In [4]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [5]:
words = open('names.txt', 'r').read().splitlines()

In [6]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i + 1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
vocab_size = len(itos)
batch_size = 32

In [7]:
# build data set
block_size = 3

def build_dataset(words):
    X, Y = [], []
    for w in words:
        context = [0] * block_size # padding with [0] = '.'
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix) 
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y
import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))
Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

In [8]:
# pytorch gradients
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact" {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [9]:
n_embd = 10
n_hidden = 64

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd), generator=g)
# layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size) ** 0.5)
b1 = torch.randn(n_hidden, generator=g) * 0.1
# layer 2
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden)) *0.1 + 1.0
bnbias = torch.randn((1, n_hidden)) * 0.1

parameters = [C, W1, b1, W2, b2, bnbias, bngain]
for p in parameters:
    p.requires_grad = True

In [10]:
# constructing mini batch
ix = torch.randint(0, Xtr.shape[0], (batch_size, ), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

# forward pass

emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)

# Linear layer 1

hprebn = embcat @ W1 + b1
# Batchnorm layer

bnmeani = 1/batch_size*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(batch_size-1)*(bndiff2).sum(0, keepdim=True)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias

# Non-linearity

h = torch.tanh(hpreact)

# Linear layer 2

logits = h @ W2 + b2

# Cross entropy loss

logits_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logits_maxes
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[(range(batch_size)), Yb].mean()


# Pytorch backward pass
for p in parameters:
    p.grad = None

for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, norm_logits, logits_maxes, logits, h, hpreact, bnvar_inv,
          bnvar, bnraw, bndiff2, bndiff, hprebn, bnmeani, hprebn, embcat, emb]:
    t.retain_grad()

loss.backward()
loss

tensor(3.3297, grad_fn=<NegBackward0>)

In [13]:
# Manual Backprop here

dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(batch_size), Yb] = -1.0 / batch_size # y = -(a + b + c).. d/dy = -1/3 for each, derivative for all others in 32, 27 are 0 since they don't affect loss

dprobs = (1.0/probs) * dlogprobs # d/dx logx = 1/x, then multiply by dlogprob (chain rule)

dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)
dcounts = counts_sum_inv * dprobs # since c = a* b, and we want d/da, its equal to b, then chain rule so dprobs
dcounts_sum = (-counts_sum ** -2) * dcounts_sum_inv # power rule plus chain rule
dcounts += torch.ones_like(counts) * dcounts_sum # replicates the 27 by 1 tensor into 32 by 27.
dnorm_logits = (counts) * dcounts # derivative of e^x is e^x, then chain rule to add dcounts
dlogits = dnorm_logits.clone()
dlogits_maxes = (-dnorm_logits).sum(1, keepdim=True) # should be near zero, since it barely impacts the loss
dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogits_maxes # makes a onehot matrix that has 1 in indices that has a max value from logits
dh = dlogits @ W2.T
dW2 = h.T @ dlogits
db2 = dlogits.sum(0)
dhpreact = (1.0 - h**2) *dh
dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
dbnraw = (bngain * dhpreact)
dbnbias = dhpreact.sum(0, keepdim=True)
dbndiff = bnvar_inv * dbnraw
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
dbnvar = (-0.5*(bnvar+1e-5)**-1.5) * dbnvar_inv
dbndiff2 = (1.0 / (batch_size-1)) * torch.ones_like(bndiff2) * dbnvar
dbndiff += 2 * dbndiff * bndiff2
dhprebn = dbndiff.clone()
dbnmeani = (-dbndiff).sum(0)
dhprebn += torch.ones_like(hprebn) * (1.0 / batch_size) * dbnmeani
dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)
demb = dembcat.view(emb.shape)

cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogits_maxes, logits_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
# cmp('C', dC, C)


logprobs        | exact" True  | approximate: True  | maxdiff: 0.0
probs           | exact" True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact" True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact" True  | approximate: True  | maxdiff: 0.0
counts          | exact" True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact" True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact" True  | approximate: True  | maxdiff: 0.0
logits          | exact" True  | approximate: True  | maxdiff: 0.0
h               | exact" True  | approximate: True  | maxdiff: 0.0


NameError: name 'dW2' is not defined